# 타이타닉 생존자 예측 - Pro_3

## 🎯 Pro_3 핵심 전략
1. **KNN Imputer**: n_neighbors 최적화
2. **Fare → Fare_Per_Person**
3. **아이 정의**: Age < 10세
4. **WCG**: 가족 내 여성+아이 생존 → Master도 생존
5. **과적합 자동 조정**: Optuna 튜닝 후 과적합 시 재조정
6. **RepeatedStratifiedKFold**: n_repeats=10으로 안정적인 CV

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.model_selection import train_test_split, StratifiedKFold, RepeatedStratifiedKFold, cross_val_score
from sklearn.impute import KNNImputer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import optuna
from optuna.samplers import TPESampler
import warnings
warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)
print('✅ 라이브러리 임포트 완료')

✅ 라이브러리 임포트 완료


In [39]:
base_path = r'C:\Users\user\github\DataScience\scikit-learn\scikit-learn\data\titanic'
train_df = pd.read_csv(f'{base_path}/train.csv')
test_df = pd.read_csv(f'{base_path}/test.csv')
test_passenger_ids = test_df['PassengerId'].copy()
print(f'Train: {train_df.shape}, Test: {test_df.shape}')

Train: (891, 12), Test: (418, 11)


In [40]:
def prepare_for_knn(df):
    df = df.copy()
    df['Sex'] = (df['Sex'] == 'male').astype(int)
    df['Embarked'] = df['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).fillna(-1)
    return df

def apply_knn_imputer(df, n_neighbors=5):
    imputer_cols = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
    imputer = KNNImputer(n_neighbors=n_neighbors)
    df_imputed = pd.DataFrame(imputer.fit_transform(df[imputer_cols]), columns=imputer_cols, index=df.index)
    df['Age'] = df_imputed['Age']
    df['Fare'] = df_imputed['Fare']
    df['Embarked'] = df_imputed['Embarked'].round().astype(int)
    return df

def find_best_k_neighbors(train_df):
    print('🔍 KNN Imputer n_neighbors 최적화 중...')
    results = []
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)
    for k in range(3, 16, 2):
        df = prepare_for_knn(train_df.copy())
        df = apply_knn_imputer(df, n_neighbors=k)
        X = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
        y = df['Survived'].astype(int)
        model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42, n_jobs=-1)
        scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy')
        results.append({'k': k, 'cv_accuracy': scores.mean()})
        print(f'  k={k}: CV Accuracy = {scores.mean():.4f}')
    results_df = pd.DataFrame(results)
    best_k = results_df.loc[results_df['cv_accuracy'].idxmax(), 'k']
    print(f'\n🏆 최적 n_neighbors = {best_k}')
    return int(best_k)

best_k = find_best_k_neighbors(train_df)

🔍 KNN Imputer n_neighbors 최적화 중...
  k=3: CV Accuracy = 0.8211
  k=5: CV Accuracy = 0.8221
  k=7: CV Accuracy = 0.8232
  k=9: CV Accuracy = 0.8226
  k=11: CV Accuracy = 0.8212
  k=13: CV Accuracy = 0.8231
  k=15: CV Accuracy = 0.8217

🏆 최적 n_neighbors = 7


In [41]:
def add_wcg_family_survival(all_data, train_len, best_k):
    all_data['Last_Name'] = all_data['Name'].apply(lambda x: x.split(',')[0])
    all_data['Title'] = all_data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    all_data['TicketFrequency'] = all_data['Ticket'].map(all_data['Ticket'].value_counts())
    all_data['Sex_num'] = (all_data['Sex'] == 'male').astype(int)
    all_data['Embarked_num'] = all_data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
    print(f'\n🔧 KNN Imputer 적용 (n_neighbors={best_k})...')
    imputer_cols = ['Pclass', 'Sex_num', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_num']
    imputer = KNNImputer(n_neighbors=best_k)
    df_imputed = pd.DataFrame(imputer.fit_transform(all_data[imputer_cols]), columns=imputer_cols, index=all_data.index)
    all_data['Age'] = df_imputed['Age']
    all_data['Fare'] = df_imputed['Fare']
    all_data['Embarked_num'] = df_imputed['Embarked_num'].round().astype(int)
    all_data['Fare_Per_Person'] = all_data['Fare'] / all_data['TicketFrequency']
    all_data['Fare_Per_Person_Round'] = all_data['Fare_Per_Person'].round(2)
    all_data['IsChild'] = (all_data['Age'] < 10).astype(int)
    all_data['IsFemale'] = (all_data['Sex'] == 'female').astype(int)
    all_data['IsMaster'] = (all_data['Title'] == 'Master').astype(int)
    all_data['IsMaleChild'] = ((all_data['Sex'] == 'male') & (all_data['IsChild'] == 1)).astype(int)
    all_data['Family_Survival'] = 0.5
    all_data['WCG_Survival'] = 0
    print('📊 Family_Survival 계산 중...')
    for ticket, grp_df in all_data.groupby('Ticket'):
        if len(grp_df) > 1:
            for idx in grp_df.index:
                others = grp_df.drop(idx)
                others_train = others[others.index < train_len]
                if len(others_train) > 0:
                    if others_train['Survived'].max() == 1.0:
                        all_data.loc[idx, 'Family_Survival'] = 1
                    elif others_train['Survived'].min() == 0.0:
                        all_data.loc[idx, 'Family_Survival'] = 0
    for (last_name, fare_pp, embarked), grp_df in all_data.groupby(['Last_Name', 'Fare_Per_Person_Round', 'Embarked_num']):
        if len(grp_df) > 1:
            for idx in grp_df.index:
                if all_data.loc[idx, 'Family_Survival'] == 0.5:
                    others = grp_df.drop(idx)
                    others_train = others[others.index < train_len]
                    if len(others_train) > 0:
                        if others_train['Survived'].max() == 1.0:
                            all_data.loc[idx, 'Family_Survival'] = 1
                        elif others_train['Survived'].min() == 0.0:
                            all_data.loc[idx, 'Family_Survival'] = 0
    print('🎯 WCG 전략 적용 중...')
    for ticket, grp_df in all_data.groupby('Ticket'):
        if len(grp_df) > 1:
            grp_train = grp_df[grp_df.index < train_len]
            if len(grp_train) > 0:
                women_children = grp_train[(grp_train['IsFemale'] == 1) | (grp_train['IsChild'] == 1)]
                if len(women_children) > 0 and (women_children['Survived'] == 1).all():
                    for idx in grp_df.index:
                        if all_data.loc[idx, 'IsMaster'] == 1 or all_data.loc[idx, 'IsMaleChild'] == 1:
                            all_data.loc[idx, 'WCG_Survival'] = 1
                            all_data.loc[idx, 'Family_Survival'] = 1
    print(f'✅ Family_Survival 분포: {all_data["Family_Survival"].value_counts().to_dict()}')
    return all_data

all_data = pd.concat([train_df, test_df], ignore_index=True)
train_len = len(train_df)
all_data = add_wcg_family_survival(all_data, train_len, best_k)
print('\n✅ KNN Imputer + WCG 특성 생성 완료')


🔧 KNN Imputer 적용 (n_neighbors=7)...
📊 Family_Survival 계산 중...
🎯 WCG 전략 적용 중...
✅ Family_Survival 분포: {0.5: 754, 1.0: 309, 0.0: 246}

✅ KNN Imputer + WCG 특성 생성 완료


In [42]:
def preprocessing_pro3(all_data, train_len):
    title_mapping = {'Mr': 'Mr', 'Miss': 'Miss', 'Mrs': 'Mrs', 'Master': 'Master',
        'Dr': 'Rare', 'Rev': 'Rare', 'Col': 'Rare', 'Major': 'Rare', 'Capt': 'Rare',
        'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs',
        'Lady': 'Rare', 'Countess': 'Rare', 'Sir': 'Rare', 'Don': 'Rare', 'Dona': 'Rare', 'Jonkheer': 'Rare'}
    all_data['Title'] = all_data['Title'].map(title_mapping).fillna('Rare')
    all_data['Sex'] = (all_data['Sex'] == 'male').astype(int)
    all_data['FamilySize'] = all_data['SibSp'] + all_data['Parch'] + 1
    all_data['IsAlone'] = (all_data['FamilySize'] == 1).astype(int)
    def age_to_bin(age):
        if pd.isna(age): return 2
        elif age < 10: return 0
        elif age < 18: return 1
        elif age < 35: return 2
        elif age < 50: return 3
        else: return 4
    all_data['AgeBin'] = all_data['Age'].apply(age_to_bin).astype(int)
    all_data['LogFare'] = np.log1p(all_data['Fare_Per_Person'])
    all_data['Deck'] = all_data['Cabin'].str[0].fillna('U')
    deck_map = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'T': 0, 'U': 0}
    all_data['Deck'] = all_data['Deck'].map(deck_map).fillna(0).astype(int)
    title_enc = {'Mr': 0, 'Miss': 1, 'Mrs': 2, 'Master': 3, 'Rare': 4}
    all_data['Title'] = all_data['Title'].map(title_enc).fillna(4).astype(int)
    all_data['Sex_Pclass'] = all_data['Sex'] * all_data['Pclass']
    features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare_Per_Person', 'LogFare',
                'Embarked_num', 'Title', 'FamilySize', 'IsAlone', 'IsChild', 'IsMaster',
                'IsMaleChild', 'Family_Survival', 'WCG_Survival', 'TicketFrequency',
                'Deck', 'AgeBin', 'Sex_Pclass', 'Survived']
    all_data = all_data[features]
    for col in all_data.columns:
        if all_data[col].dtype == 'category':
            all_data[col] = all_data[col].astype(int)
    print(f'✅ 전처리 완료, 특성 수: {len(features) - 1}')
    return all_data

all_data = preprocessing_pro3(all_data, train_len)
all_data.head()

✅ 전처리 완료, 특성 수: 20


,Pclass,Sex,Age,SibSp,Parch,Fare_Per_Person,LogFare,Embarked_num,Title,FamilySize,...,IsChild,IsMaster,IsMaleChild,Family_Survival,WCG_Survival,TicketFrequency,Deck,AgeBin,Sex_Pclass,Survived
0,3,1,22.0,1,0,7.25000,2.110213,0,0,2,...,0,0,0,0.5,0,1,0,2,3,0.0
1,1,0,38.0,1,0,35.64165,3.601186,1,2,2,...,0,0,0,0.5,0,2,3,3,0,1.0
2,3,0,26.0,0,0,7.92500,2.188856,0,1,1,...,0,0,0,0.5,0,1,0,2,0,1.0
3,1,0,35.0,1,0,26.55000,3.316003,0,2,2,...,0,0,0,0.0,0,2,3,3,0,1.0
4,3,1,35.0,0,0,8.05000,2.202765,0,0,1,...,0,0,0,0.5,0,1,0,3,3,0.0


In [43]:
train_processed = all_data.iloc[:train_len].copy()
test_processed = all_data.iloc[train_len:].copy()
X = train_processed.drop('Survived', axis=1)
y = train_processed['Survived'].astype(int)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

# RepeatedStratifiedKFold 사용 (n_repeats=10)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)
cv_optuna = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)

print(f'Train: {X_train.shape}, Validation: {X_val.shape}')
print(f'CV: RepeatedStratifiedKFold(n_splits=5, n_repeats=10) → 총 50회')

Train: (757, 20), Validation: (134, 20)
CV: RepeatedStratifiedKFold(n_splits=5, n_repeats=10) → 총 50회


In [44]:
def check_overfitting_with_feedback(model, X_train, y_train, model_name, cv, threshold=0.05):
    model_clone = model.__class__(**model.get_params())
    model_clone.fit(X_train, y_train)
    train_acc = model_clone.score(X_train, y_train)
    cv_scores = cross_val_score(model_clone, X_train, y_train, cv=cv, scoring='accuracy')
    cv_acc = cv_scores.mean()
    gap = train_acc - cv_acc
    is_overfitting = gap > threshold
    if gap > 0.10:
        print(f'🚨 {model_name}: 심각한 과적합! Train={train_acc:.4f}, CV={cv_acc:.4f}, Gap={gap:.4f}')
    elif gap > 0.05:
        print(f'⚠️ {model_name}: 과적합 경고. Train={train_acc:.4f}, CV={cv_acc:.4f}, Gap={gap:.4f}')
    else:
        print(f'✅ {model_name}: 정상. Train={train_acc:.4f}, CV={cv_acc:.4f}, Gap={gap:.4f}')
    return is_overfitting, gap, train_acc, cv_acc

print('과적합 감지 함수 정의 완료')

과적합 감지 함수 정의 완료


In [ ]:
class AdaptiveHyperparams:
    def __init__(self):
        self.reset()
        
    def reset(self):
        # 초기값은 기존과 동일하게 유지 (처음엔 자유롭게 탐색)
        self.max_depth_range = (2, 5)
        self.reg_alpha_min = 0.1
        self.reg_lambda_min = 0.1
        self.subsample_range = (0.6, 0.9)
        self.adjustment_count = 0
        
    def adjust_for_overfitting(self):
        self.adjustment_count += 1
        print(f'\n💊 [강력 처방] 과적합 긴급 조정 #{self.adjustment_count}')
        
        # 1. Depth (트리 깊이): 과적합의 주범
        # [수정] 1씩 줄이는 게 아니라, 바로 상한선을 3으로 깎아버립니다.
        # 깊이가 3만 되어도 타이타닉 데이터 패턴은 충분히 배웁니다.
        new_max = max(2, self.max_depth_range[1] - 2) 
        self.max_depth_range = (2, new_max)
        print(f'   📉 max_depth: {self.max_depth_range} (깊이를 과감히 제한)')
        
        # 2. Regularization (규제): 가장 큰 변경점
        # [수정] 2배씩 늘리면 너무 느립니다. 10배씩 늘려서 노이즈를 아예 무시하게 만듭니다.
        # 0.1 -> 1.0 -> 10.0 순으로 급격히 강력해집니다.
        self.reg_alpha_min = min(max(self.reg_alpha_min * 10, 1.0), 20.0)
        self.reg_lambda_min = min(max(self.reg_lambda_min * 10, 1.0), 20.0)
        print(f'   🛡️ reg_min: {self.reg_alpha_min} (규제 강도 10배 강화)')
        
        # 3. Subsample (데이터 샘플링):
        # [수정] 데이터를 덜 보게 하여 암기를 방지합니다. 상한선을 0.2씩 팍팍 낮춥니다.
        new_upper = max(0.5, self.subsample_range[1] - 0.2)
        new_lower = max(0.4, self.subsample_range[0] - 0.1)
        self.subsample_range = (new_lower, new_upper)
        print(f'   👀 subsample: {self.subsample_range} (데이터 노출 대폭 축소)')
        
        # 3번 정도의 강력한 처방이면 충분합니다. (5번까지 질질 끌지 않음)
        return self.adjustment_count < 3

adaptive_params = AdaptiveHyperparams()
print('✅ AdaptiveHyperparams (강력 처방 Ver.) 재정의 완료')

AdaptiveHyperparams 클래스 정의 완료


In [46]:
def tune_xgb(X_train, y_train, cv, adaptive_params, max_iterations=5):
    adaptive_params.reset()
    for iteration in range(max_iterations):
        print(f'\n{"="*50}\n🚀 XGBoost 튜닝 시도 #{iteration + 1}\n{"="*50}')
        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 300),
                'max_depth': trial.suggest_int('max_depth', adaptive_params.max_depth_range[0], adaptive_params.max_depth_range[1]),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
                'subsample': trial.suggest_float('subsample', adaptive_params.subsample_range[0], adaptive_params.subsample_range[1]),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.8),
                'reg_alpha': trial.suggest_float('reg_alpha', adaptive_params.reg_alpha_min, 10.0, log=True),
                'reg_lambda': trial.suggest_float('reg_lambda', adaptive_params.reg_lambda_min, 10.0, log=True),
                'random_state': 42, 'use_label_encoder': False, 'eval_metric': 'logloss', 'n_jobs': -1
            }
            model = XGBClassifier(**params)
            return cross_val_score(model, X_train, y_train, cv=cv_optuna, scoring='accuracy').mean()
        study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42+iteration))
        study.optimize(objective, n_trials=30, timeout=120, show_progress_bar=True)
        print(f'\n🏆 Best CV: {study.best_value:.4f}')
        best_model = XGBClassifier(**study.best_params, random_state=42, use_label_encoder=False, eval_metric='logloss', n_jobs=-1)
        best_model.fit(X_train, y_train)
        is_overfitting, gap, train_acc, cv_acc = check_overfitting_with_feedback(best_model, X_train, y_train, 'XGBoost', cv)
        if not is_overfitting:
            return study, best_model
        if not adaptive_params.adjust_for_overfitting():
            return study, best_model
    return study, best_model

study_xgb, best_xgb = tune_xgb(X_train, y_train, cv, adaptive_params)


🚀 XGBoost 튜닝 시도 #1


Best trial: 24. Best value: 0.872254: 100%|██████████| 30/30 [01:20<00:00,  2.69s/it, 80.62/120 seconds]



🏆 Best CV: 0.8723
✅ XGBoost: 정상. Train=0.8956, CV=0.8723, Gap=0.0234


In [47]:
def tune_lgbm(X_train, y_train, cv, adaptive_params, max_iterations=5):
    adaptive_params.reset()
    for iteration in range(max_iterations):
        print(f'\n{"="*50}\n🚀 LightGBM 튜닝 시도 #{iteration + 1}\n{"="*50}')
        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 300),
                'max_depth': trial.suggest_int('max_depth', adaptive_params.max_depth_range[0], adaptive_params.max_depth_range[1]),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
                'subsample': trial.suggest_float('subsample', adaptive_params.subsample_range[0], adaptive_params.subsample_range[1]),
                'reg_alpha': trial.suggest_float('reg_alpha', adaptive_params.reg_alpha_min, 10.0, log=True),
                'reg_lambda': trial.suggest_float('reg_lambda', adaptive_params.reg_lambda_min, 10.0, log=True),
                'num_leaves': trial.suggest_int('num_leaves', 8, 31),
                'random_state': 42, 'verbose': -1, 'n_jobs': -1
            }
            model = LGBMClassifier(**params)
            return cross_val_score(model, X_train, y_train, cv=cv_optuna, scoring='accuracy').mean()
        study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42+iteration))
        study.optimize(objective, n_trials=30, timeout=120, show_progress_bar=True)
        print(f'\n🏆 Best CV: {study.best_value:.4f}')
        best_model = LGBMClassifier(**study.best_params, random_state=42, verbose=-1, n_jobs=-1)
        best_model.fit(X_train, y_train)
        is_overfitting, gap, train_acc, cv_acc = check_overfitting_with_feedback(best_model, X_train, y_train, 'LightGBM', cv)
        if not is_overfitting:
            return study, best_model
        if not adaptive_params.adjust_for_overfitting():
            return study, best_model
    return study, best_model

study_lgbm, best_lgbm = tune_lgbm(X_train, y_train, cv, adaptive_params)


🚀 LightGBM 튜닝 시도 #1


Best trial: 19. Best value: 0.869088: 100%|██████████| 30/30 [00:40<00:00,  1.36s/it, 40.85/120 seconds]



🏆 Best CV: 0.8691
✅ LightGBM: 정상. Train=0.8838, CV=0.8691, Gap=0.0147


In [48]:
def tune_catboost(X_train, y_train, cv, adaptive_params, max_iterations=5):
    adaptive_params.reset()
    for iteration in range(max_iterations):
        print(f'\n{"="*50}\n🚀 CatBoost 튜닝 시도 #{iteration + 1}\n{"="*50}')
        def objective(trial):
            params = {
                'iterations': trial.suggest_int('iterations', 100, 300),
                'depth': trial.suggest_int('depth', adaptive_params.max_depth_range[0], adaptive_params.max_depth_range[1]),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
                'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', adaptive_params.reg_lambda_min, 10.0),
                'random_seed': 42, 'verbose': 0
            }
            scores = []
            for train_idx, val_idx in StratifiedKFold(n_splits=5, shuffle=True, random_state=42).split(X_train, y_train):
                model = CatBoostClassifier(**params)
                model.fit(X_train.iloc[train_idx], y_train.iloc[train_idx])
                scores.append(accuracy_score(y_train.iloc[val_idx], model.predict(X_train.iloc[val_idx])))
            return np.mean(scores)
        study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42+iteration))
        study.optimize(objective, n_trials=20, timeout=120, show_progress_bar=True)
        print(f'\n🏆 Best CV: {study.best_value:.4f}')
        best_model = CatBoostClassifier(**study.best_params, random_seed=42, verbose=0)
        best_model.fit(X_train, y_train)
        train_acc = best_model.score(X_train, y_train)
        cv_scores = cross_val_score(CatBoostClassifier(**study.best_params, random_seed=42, verbose=0), X_train, y_train, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), scoring='accuracy')
        cv_acc = cv_scores.mean()
        gap = train_acc - cv_acc
        is_overfitting = gap > 0.05
        if gap > 0.10:
            print(f'🚨 CatBoost: 심각한 과적합! Train={train_acc:.4f}, CV={cv_acc:.4f}, Gap={gap:.4f}')
        elif gap > 0.05:
            print(f'⚠️ CatBoost: 과적합 경고. Train={train_acc:.4f}, CV={cv_acc:.4f}, Gap={gap:.4f}')
        else:
            print(f'✅ CatBoost: 정상. Train={train_acc:.4f}, CV={cv_acc:.4f}, Gap={gap:.4f}')
        if not is_overfitting:
            return study, best_model
        if not adaptive_params.adjust_for_overfitting():
            return study, best_model
    return study, best_model

study_catboost, best_catboost = tune_catboost(X_train, y_train, cv, adaptive_params)


🚀 CatBoost 튜닝 시도 #1


Best trial: 17. Best value: 0.870486: 100%|██████████| 20/20 [00:18<00:00,  1.07it/s, 18.68/120 seconds]



🏆 Best CV: 0.8705
✅ CatBoost: 정상. Train=0.8970, CV=0.8705, Gap=0.0265


In [49]:
def tune_rf(X_train, y_train, cv, adaptive_params, max_iterations=5):
    adaptive_params.reset()
    for iteration in range(max_iterations):
        print(f'\n{"="*50}\n🚀 RandomForest 튜닝 시도 #{iteration + 1}\n{"="*50}')
        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 300),
                'max_depth': trial.suggest_int('max_depth', adaptive_params.max_depth_range[0], max(adaptive_params.max_depth_range[1], 5)),
                'min_samples_split': trial.suggest_int('min_samples_split', 5, 20),
                'min_samples_leaf': trial.suggest_int('min_samples_leaf', 3, 10),
                'random_state': 42, 'n_jobs': -1
            }
            model = RandomForestClassifier(**params)
            return cross_val_score(model, X_train, y_train, cv=cv_optuna, scoring='accuracy').mean()
        study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42+iteration))
        study.optimize(objective, n_trials=30, timeout=90, show_progress_bar=True)
        print(f'\n🏆 Best CV: {study.best_value:.4f}')
        best_model = RandomForestClassifier(**study.best_params, random_state=42, n_jobs=-1)
        best_model.fit(X_train, y_train)
        is_overfitting, gap, train_acc, cv_acc = check_overfitting_with_feedback(best_model, X_train, y_train, 'RandomForest', cv)
        if not is_overfitting:
            return study, best_model
        if not adaptive_params.adjust_for_overfitting():
            return study, best_model
    return study, best_model

study_rf, best_rf = tune_rf(X_train, y_train, cv, adaptive_params)


🚀 RandomForest 튜닝 시도 #1


Best trial: 10. Best value: 0.864195:  40%|████      | 12/30 [01:37<02:25,  8.10s/it, 97.21/90 seconds]



🏆 Best CV: 0.8642
✅ RandomForest: 정상. Train=0.8824, CV=0.8642, Gap=0.0182


In [50]:
models = {'XGBoost': best_xgb, 'LightGBM': best_lgbm, 'CatBoost': best_catboost, 'RandomForest': best_rf}
print('\n📊 Validation Set 성능:')
print('='*50)
for name, model in models.items():
    acc = accuracy_score(y_val, model.predict(X_val))
    print(f'{name}: {acc:.4f}')


📊 Validation Set 성능:
XGBoost: 0.7761
LightGBM: 0.7985
CatBoost: 0.7836
RandomForest: 0.7910


In [51]:
acc_xgb = accuracy_score(y_val, best_xgb.predict(X_val))
acc_lgbm = accuracy_score(y_val, best_lgbm.predict(X_val))
acc_cat = accuracy_score(y_val, best_catboost.predict(X_val))
acc_rf = accuracy_score(y_val, best_rf.predict(X_val))
total = acc_xgb + acc_lgbm + acc_cat + acc_rf
weights = [acc_xgb/total, acc_lgbm/total, acc_cat/total, acc_rf/total]

print('\n' + '='*60)
print('📊 Soft Voting vs Hard Voting 비교')
print('='*60)

voting_soft = VotingClassifier(estimators=[('xgb', best_xgb), ('lgbm', best_lgbm), ('cat', best_catboost), ('rf', best_rf)], voting='soft', weights=weights)
voting_soft.fit(X_train, y_train)
soft_acc = accuracy_score(y_val, voting_soft.predict(X_val))
print(f'\n🔵 Soft Voting (Weighted): {soft_acc:.4f}')

voting_hard = VotingClassifier(estimators=[('xgb', best_xgb), ('lgbm', best_lgbm), ('cat', best_catboost), ('rf', best_rf)], voting='hard')
voting_hard.fit(X_train, y_train)
hard_acc = accuracy_score(y_val, voting_hard.predict(X_val))
print(f'🔴 Hard Voting:            {hard_acc:.4f}')

stacking = StackingClassifier(estimators=[('xgb', best_xgb), ('lgbm', best_lgbm), ('cat', best_catboost), ('rf', best_rf)], final_estimator=LogisticRegression(C=0.1, max_iter=1000), cv=5, n_jobs=-1)
stacking.fit(X_train, y_train)
stacking_acc = accuracy_score(y_val, stacking.predict(X_val))
print(f'🟢 Stacking:               {stacking_acc:.4f}')


📊 Soft Voting vs Hard Voting 비교

🔵 Soft Voting (Weighted): 0.7836
🔴 Hard Voting:            0.7910
🟢 Stacking:               0.7836


In [52]:
import os
X_full = train_processed.drop('Survived', axis=1)
y_full = train_processed['Survived'].astype(int)

final_xgb = XGBClassifier(**study_xgb.best_params, random_state=42, use_label_encoder=False, eval_metric='logloss', n_jobs=-1)
final_lgbm = LGBMClassifier(**study_lgbm.best_params, random_state=42, verbose=-1, n_jobs=-1)
final_catboost = CatBoostClassifier(**study_catboost.best_params, random_seed=42, verbose=0)
final_rf = RandomForestClassifier(**study_rf.best_params, random_state=42, n_jobs=-1)

final_xgb.fit(X_full, y_full)
final_lgbm.fit(X_full, y_full)
final_catboost.fit(X_full, y_full)
final_rf.fit(X_full, y_full)

final_voting_soft = VotingClassifier(estimators=[('xgb', final_xgb), ('lgbm', final_lgbm), ('cat', final_catboost), ('rf', final_rf)], voting='soft', weights=weights)
final_voting_soft.fit(X_full, y_full)

final_voting_hard = VotingClassifier(estimators=[('xgb', final_xgb), ('lgbm', final_lgbm), ('cat', final_catboost), ('rf', final_rf)], voting='hard')
final_voting_hard.fit(X_full, y_full)

final_stacking = StackingClassifier(estimators=[('xgb', final_xgb), ('lgbm', final_lgbm), ('cat', final_catboost), ('rf', final_rf)], final_estimator=LogisticRegression(C=0.1, max_iter=1000), cv=5, n_jobs=-1)
final_stacking.fit(X_full, y_full)

print('✅ 전체 데이터로 재학습 완료')

✅ 전체 데이터로 재학습 완료


In [53]:
X_test = test_processed.drop('Survived', axis=1, errors='ignore')
output_path = r'C:\Users\user\github\DataScience\scikit-learn\scikit-learn\Submission'
os.makedirs(output_path, exist_ok=True)

models_for_submission = {
    'Pro3_XGBoost': final_xgb, 'Pro3_LightGBM': final_lgbm, 'Pro3_CatBoost': final_catboost, 'Pro3_RandomForest': final_rf,
    'Pro3_SoftVoting': final_voting_soft, 'Pro3_HardVoting': final_voting_hard, 'Pro3_Stacking': final_stacking
}

print('\n📂 CSV 저장 중...')
for name, model in models_for_submission.items():
    predictions = model.predict(X_test).astype(int)
    submission = pd.DataFrame({'PassengerId': test_passenger_ids, 'Survived': predictions})
    file_path = f'{output_path}/submission_{name}.csv'
    submission.to_csv(file_path, index=False)
    print(f'✅ {file_path} (생존: {predictions.sum()}/{len(predictions)})')

print('\n🎉 완료!')


📂 CSV 저장 중...
✅ C:\Users\user\github\DataScience\scikit-learn\scikit-learn\Submission/submission_Pro3_XGBoost.csv (생존: 150/418)
✅ C:\Users\user\github\DataScience\scikit-learn\scikit-learn\Submission/submission_Pro3_LightGBM.csv (생존: 153/418)
✅ C:\Users\user\github\DataScience\scikit-learn\scikit-learn\Submission/submission_Pro3_CatBoost.csv (생존: 149/418)
✅ C:\Users\user\github\DataScience\scikit-learn\scikit-learn\Submission/submission_Pro3_RandomForest.csv (생존: 144/418)
✅ C:\Users\user\github\DataScience\scikit-learn\scikit-learn\Submission/submission_Pro3_SoftVoting.csv (생존: 152/418)
✅ C:\Users\user\github\DataScience\scikit-learn\scikit-learn\Submission/submission_Pro3_HardVoting.csv (생존: 147/418)
✅ C:\Users\user\github\DataScience\scikit-learn\scikit-learn\Submission/submission_Pro3_Stacking.csv (생존: 151/418)

🎉 완료!


In [54]:
final_results = [
    {'Model': 'XGBoost', 'CV_Acc': study_xgb.best_value, 'Val_Acc': acc_xgb},
    {'Model': 'LightGBM', 'CV_Acc': study_lgbm.best_value, 'Val_Acc': acc_lgbm},
    {'Model': 'CatBoost', 'CV_Acc': study_catboost.best_value, 'Val_Acc': acc_cat},
    {'Model': 'RandomForest', 'CV_Acc': study_rf.best_value, 'Val_Acc': acc_rf},
    {'Model': 'SoftVoting', 'CV_Acc': None, 'Val_Acc': soft_acc},
    {'Model': 'HardVoting', 'CV_Acc': None, 'Val_Acc': hard_acc},
    {'Model': 'Stacking', 'CV_Acc': None, 'Val_Acc': stacking_acc}
]

final_df = pd.DataFrame(final_results).sort_values('Val_Acc', ascending=False)
print('\n' + '='*60)
print('📊 최종 성능 요약 (RepeatedStratifiedKFold n_repeats=10)')
print('='*60)
print(final_df.to_string(index=False))


📊 최종 성능 요약 (RepeatedStratifiedKFold n_repeats=10)
       Model   CV_Acc  Val_Acc
    LightGBM 0.869088 0.798507
  HardVoting      NaN 0.791045
RandomForest 0.864195 0.791045
  SoftVoting      NaN 0.783582
    CatBoost 0.870486 0.783582
    Stacking      NaN 0.783582
     XGBoost 0.872254 0.776119


In [55]:
feature_importance = pd.DataFrame({'Feature': X_full.columns, 'Importance': final_xgb.feature_importances_}).sort_values('Importance', ascending=False)
fig = px.bar(feature_importance, x='Importance', y='Feature', orientation='h', title='Feature Importance (XGBoost)', color='Importance')
fig.update_layout(yaxis={'categoryorder': 'total ascending'})
fig.show()
print('\n📌 주요 특성:')
print(feature_importance.to_string(index=False))


📌 주요 특성:
        Feature  Importance
            Sex    0.170686
     Sex_Pclass    0.156553
          Title    0.153220
Family_Survival    0.094792
         Pclass    0.075879
       IsMaster    0.069825
   WCG_Survival    0.065001
     FamilySize    0.027053
        LogFare    0.026530
Fare_Per_Person    0.022529
TicketFrequency    0.022014
           Deck    0.021906
            Age    0.015993
   Embarked_num    0.014768
          SibSp    0.014369
         AgeBin    0.012849
          Parch    0.012407
        IsAlone    0.010270
    IsMaleChild    0.009997
        IsChild    0.003360
